# Controlling Agent Reasoning Loop with `return_direct` in Tools

## SETUP

In [1]:
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
ELEVENLABS_API_KEY = os.getenv("ELEVENLABS_API_KEY")
MISTRALAI_API_KEY = os.getenv("MISTRALAI_API_KEY")
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

In [3]:
from typing import Optional

from llama_index.llms.openai import OpenAI
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner
from llama_index.core.bridge.pydantic import BaseModel

In [4]:
llm_openai = OpenAI(api_key=OPENAI_API_KEY, model="gpt-4o")

## BOOKING TOOLS

In [16]:
bookings = {}


class Booking(BaseModel):
    name: Optional[str] = None
    email: Optional[str] = None
    phone: Optional[str] = None
    date: Optional[str] = None
    time: Optional[str] = None


def get_booking_state(user_id: str) -> str:
    """Get the current state of a booking for a given booking ID."""
    try:
        return str(bookings[user_id].dict())
    except:
        return f"Booking ID {user_id} not found"


def update_booking(user_id: str, property: str, value: str) -> str:
    """Update a property of a booking for a given booking ID. Only enter details that are explicitly provided."""
    booking = bookings[user_id]
    setattr(booking, property, value)
    return f"Booking ID {user_id} updated with {property} = {value}"


def create_booking(user_id: str) -> str:
    """Create a new booking and return the booking ID."""
    bookings[user_id] = Booking()
    return "Booking created, but not yet confirmed. Please provide your name, email, phone, date, & time."


def confirm_booking(user_id: str) -> str:
    """Confirm a booking for a given booking ID."""
    booking = bookings[user_id]

    if booking.name is None:
        raise ValueError("Please provide your name.")

    if booking.email is None:
        raise ValueError("Please provide your email.")

    if booking.phone is None:
        raise ValueError("Please provide your phone number")
    if booking.date is None:
        raise ValueError("Please provide the date of your booking ")
    if booking.time is None:
        raise ValueError("Please provide the time of your booking.")

    return f"Booking ID {user_id} confirmed!"

## EXPERIMENT 1

In [17]:
tool_get_booking_state = FunctionTool.from_defaults(fn=get_booking_state)
tool_update_booking = FunctionTool.from_defaults(fn=update_booking)
tool_create_booking = FunctionTool.from_defaults(fn=create_booking)
tool_confirm_booking = FunctionTool.from_defaults(fn=confirm_booking)

In [19]:
user = "user123"
prefix_messages = [
    ChatMessage(
        role="system",
        content=(
            f"""You are now connected to the booking system and helping the {user} with making a booking. Only enter details that the user has explicityly provided. Do not make up any details.
"""
        ),
    )
]

In [24]:
# create agent
worker_openai = FunctionCallingAgentWorker(
    tools=[
        tool_get_booking_state,
        tool_create_booking,
        tool_update_booking,
        tool_confirm_booking,
    ],
    llm=llm_openai,
    prefix_messages=prefix_messages,
    max_function_calls=10,
    allow_parallel_tool_calls=False,
    verbose=True,
)

agent_openai = AgentRunner(worker_openai)

In [25]:
response = agent_openai.chat("Please assist me with a trip booking")

Added user message to memory: Please assist me with a trip booking
=== LLM Response ===
I'd be happy to help you with your trip booking. Let's start by creating a new booking for you.

I'll need your user ID to proceed. Could you please provide that?


In [26]:
response = agent_openai.chat("Sure! My name is Raghu, my email id is test@gmail.com")

Added user message to memory: Sure! My name is Raghu, my email id is test@gmail.com
=== LLM Response ===
I need your user ID to proceed with the booking. Could you please provide that?


In [28]:
response = agent_openai.chat(
    "Cool. Phone number is 39429384923, preferred data and time are April 20th and 12PM respectively."
)

Added user message to memory: Cool. Phone number is 39429384923, preferred data and time are April 20th and 12PM respectively.
=== LLM Response ===
To proceed with your booking, I need your user ID. Could you please provide that?


In [29]:
response = agent_openai.chat("my id is A213B")

Added user message to memory: my id is A213B
=== Calling Function ===
Calling function: create_booking with args: {"user_id": "A213B"}
=== Function Output ===
Booking created, but not yet confirmed. Please provide your name, email, phone, date, & time.
=== Calling Function ===
Calling function: update_booking with args: {"user_id": "A213B", "property": "name", "value": "Raghu"}
=== Function Output ===
Booking ID A213B updated with name = Raghu
=== Calling Function ===
Calling function: update_booking with args: {"user_id": "A213B", "property": "email", "value": "test@gmail.com"}
=== Function Output ===
Booking ID A213B updated with email = test@gmail.com
=== Calling Function ===
Calling function: update_booking with args: {"user_id": "A213B", "property": "phone", "value": "39429384923"}
=== Function Output ===
Booking ID A213B updated with phone = 39429384923
=== Calling Function ===
Calling function: update_booking with args: {"user_id": "A213B", "property": "date", "value": "April 20

In [30]:
print(response)

Your booking has been successfully confirmed! Here are the details:

- **Name:** Raghu
- **Email:** test@gmail.com
- **Phone:** 39429384923
- **Date:** April 20th
- **Time:** 12PM

If you need any further assistance, feel free to ask. Have a great trip!


## EXPERIMENT 2

- return_dict = True -> response is given directly from the tool but not sent to LLM

In [33]:
tool_get_booking_state = FunctionTool.from_defaults(
    fn=get_booking_state, return_direct=True
)
tool_confirm_booking = FunctionTool.from_defaults(
    fn=confirm_booking, return_direct=True
)
tool_create_booking = FunctionTool.from_defaults(fn=create_booking, return_direct=True)
tool_update_booking = FunctionTool.from_defaults(fn=update_booking, return_direct=True)

In [34]:
user = "user123"
prefix_messages = [
    ChatMessage(
        role="system",
        content=(
            f"You are now connected to the booking system and helping {user} with making a booking. "
            "Only enter details that the user has explicitly provided. "
            "Do not make up any details."
        ),
    )
]

In [35]:
worker_openai = FunctionCallingAgentWorker(
    tools=[
        tool_confirm_booking,
        tool_create_booking,
        tool_get_booking_state,
        tool_update_booking,
    ],
    llm=llm_openai,
    prefix_messages=prefix_messages,
    allow_parallel_tool_calls=False,
    max_function_calls=10,
    verbose=True,
)

agent_openai = AgentRunner(worker_openai)

In [36]:
response = agent_openai.chat("Hello! I would like to make a booking.")

Added user message to memory: Hello! I would like to make a booking.
=== LLM Response ===
Sure! I can help you with that. Let's start by creating a new booking for you.

Could you please provide me with your user ID?


In [37]:
response = agent_openai.chat("Sure! My name is Ravi, and my email is ravi@gmail.com")

Added user message to memory: Sure! My name is Ravi, and my email is ravi@gmail.com
=== LLM Response ===
I need your user ID to proceed with the booking. Could you please provide that?


In [38]:
response = agent_openai.chat("It is A217B1")

Added user message to memory: It is A217B1
=== Calling Function ===
Calling function: create_booking with args: {"user_id": "A217B1"}
=== Function Output ===
Booking created, but not yet confirmed. Please provide your name, email, phone, date, & time.


In [39]:
response = agent_openai.chat("Ravi")

Added user message to memory: Ravi
=== Calling Function ===
Calling function: update_booking with args: {"user_id": "A217B1", "property": "name", "value": "Ravi"}
=== Function Output ===
Booking ID A217B1 updated with name = Ravi


In [40]:
response = agent_openai.chat("date June 20, 2024 @ 8:30 AM")

Added user message to memory: date June 20, 2024 @ 8:30 AM
=== Calling Function ===
Calling function: update_booking with args: {"user_id": "A217B1", "property": "date", "value": "June 20, 2024"}
=== Function Output ===
Booking ID A217B1 updated with date = June 20, 2024


In [41]:
response = agent_openai.chat("get me my booking details")

Added user message to memory: get me my booking details
=== Calling Function ===
Calling function: get_booking_state with args: {"user_id": "A217B1"}
=== Function Output ===
{'name': 'Ravi', 'email': None, 'phone': None, 'date': 'June 20, 2024', 'time': None}
